puścić od ładowania modelu

In [133]:
import gym

env = gym.make("Taxi-v3", render_mode = "ansi")
env.reset() # reset environment to a new, random state
print(env.render())
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))
print(env.s)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


Action Space Discrete(6)
State Space Discrete(500)
151


In [135]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)
env.s = state
print(env.render())

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [136]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [137]:
# brute force
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

terminated, truncated = False, False

while not truncated and not terminated:
    action = env.action_space.sample()
    state, reward, terminated, truncated, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 200
Penalties incurred: 69


In [138]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

Timestep: 200
State: 231
Action: 0
Reward: -1


In [139]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [142]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state, info = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    terminated, truncated, = False, False
    
    while not terminated and not truncated:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, terminated, truncated, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 2min 4s, sys: 18.6 s, total: 2min 22s
Wall time: 2min 17s


## stąd

In [112]:
np.savetxt('q_table.csv', q_table, delimiter=',')

In [113]:
q_table_loaded = np.loadtxt('q_table.csv', delimiter=',')

In [143]:
env = gym.make("Taxi-v3", render_mode = "ansi")
total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state, info = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    terminated, truncated = False, False
    
    while not terminated and not truncated:
        action = np.argmax(q_table_loaded[state])
        state, reward, terminated, truncated, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
env.close()

Results after 100 episodes:
Average timesteps per episode: 12.77
Average penalties per episode: 0.0


In [148]:
env = gym.make("Taxi-v3", render_mode = "human")
state, info = env.reset()
epochs, penalties, reward = 0, 0, 0

terminated, truncated = False, False

while not terminated and not truncated:
    action = np.argmax(q_table_loaded[state])
    state, reward, terminated, truncated, info = env.step(action)

    if reward == -10:
        penalties += 1

    epochs += 1

print(f"Number of timesteps: {epochs}")
print(f"Penalty: {penalties}")
env.close()

Number of timesteps: 14
Penalty: 0
